<a href="https://colab.research.google.com/github/JCoji/DataScience/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSI 4142 Assignment 1


Lucas Gavura, 300310069

Jonathan Cojita, student no

Febuary 3, 2026

# Shared code and dependency imports


In [ ]:
import seaborn as sns
import pandas as pd
import kagglehub
import os

# Dataset 2: Adult Income
https://www.kaggle.com/datasets/serpilturanyksel/adult-income

In [ ]:
#Import dataset, print preview on success
path = kagglehub.dataset_download('serpilturanyksel/adult-income')

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
if csv_files:
    csv_filename = csv_files[0]
    full_path = os.path.join(path, csv_filename)
    incomeDS = pd.read_csv(full_path)
    print(f"Successfully loaded {csv_filename} from {path}")
    display(incomeDS.head())
else:
    print(f"No CSV files found in the downloaded dataset path: {path}")
    print("Please check the contents of the downloaded folder to find the correct CSV file.")
    print(f"Contents of {path}: {os.listdir(path)}")

100%|██████████| 652k/652k [00:00<00:00, 78.2MB/s]

Extracting files...
Successfully loaded adult11.csv from /root/.cache/kagglehub/datasets/serpilturanyksel/adult-income/versions/1


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,salary
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
